In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel , AutoTokenizer , AutoModelForCausalLM
import torch
tokenizer = GPT2Tokenizer.from_pretrained("E:\\model")
model = GPT2LMHeadModel.from_pretrained("E:\\model")
if torch.cuda.is_available():
    model = model.cuda()
## utility functions ##
flatten = lambda l: [item for sublist in l for item in sublist]

def to_data(x):
    if torch.cuda.is_available():
        x = x.cpu()
    return x.data.numpy()

def to_var(x):
    if not torch.is_tensor(x):
        x = torch.Tensor(x)
    if torch.cuda.is_available():
        x = x.cuda()
    return x

def display_dialog_history(dialog_hx):
    for j, line in enumerate(dialog_hx):
        msg = tokenizer.decode(line)
        if j %2 == 0:
            print(">> User: "+ msg)
        else:
            print("Bot: "+msg)
            print()

def generate_next(bot_input_ids, do_sample=True, top_k=10, top_p=.92,
                  max_length=1000, pad_token=tokenizer.eos_token_id):
    full_msg = model.generate(bot_input_ids, do_sample=True,
                                              top_k=top_k, top_p=top_p,
                                              max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    msg = to_data(full_msg.detach()[0])[bot_input_ids.shape[-1]:]
    return msg


In [3]:
persona = [
    "Hello! Escula is a communication platform designed to help parents, students, and schools stay connected and informed about everything related to education."
]


In [4]:
# get personality facts for conversation
personas = [pers+tokenizer.eos_token for pers in persona]

personas = tokenizer.encode(''.join(['<|p2|>'] + personas + ['<|sep|>'] + ['<|start|>']))


In [5]:
import tkinter as tk

# Function to handle user input and display bot's response
def handle_input():
    user_input = user_input_entry.get()
    user_input_entry.delete(0, tk.END)

    # Encode the user input
    user_inp = tokenizer.encode(user_input + tokenizer.eos_token)
    # Append to the chat history
    dialog_hx.append(user_inp)

    # Generate a response while limiting the total chat history to 1000 tokens
    bot_input_ids = to_var([personas + flatten(dialog_hx)]).long()
    msg = generate_next(bot_input_ids)
    dialog_hx.append(msg)
    bot_response = tokenizer.decode(msg, skip_special_tokens=True)

    # Display bot's response
    chat_log.insert(tk.END, "User: {}\n".format(user_input))
    chat_log.insert(tk.END, "Bot: {}\n".format(bot_response))
    chat_log.insert(tk.END, "\n")

# Create the GUI
root = tk.Tk()
root.title("Chatbot")

# Create chat log display
chat_log = tk.Text(root, width=50, height=20)
chat_log.pack()

# Create user input entry field
user_input_entry = tk.Entry(root, width=50)
user_input_entry.pack()

# Create submit button
submit_button = tk.Button(root, text="Submit", command=handle_input)
submit_button.pack()

# Start the conversation
dialog_hx = []
root.mainloop()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
